# Assignment 2: Monte Carlo (MC) Methods in Finance

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import seaborn as sns
import pandas as pd


## Part I Option Valuation

### Q1: Monte Carlo for European Call？ Options

#### Convergence Study

In [ ]:
# Parameters
S0 = 100       # initial stock price
K = 99         # strike price
T = 1.0        # time to maturity
r = 0.06       # risk-free rate
sigma = 0.20   # volatility
M_trials = np.concatenate((np.arange(1000, 21000, 1000), np.arange(11000, 510000, 5000)))

option_prices = []
std_errors = []

# Monte Carlo simulation to estimate European Option price
def monte_carlo_simulation(S0, K, T, r, sigma, M):
    dt = T / M  # time step
    discount_factor = np.exp(-r * T)  # discount factor for present value
    
    # Generate M stock price paths
    Z = np.random.standard_normal(M)
    ST = S0 * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z) 
    payoffs = np.maximum(ST - K, 0)
    option_price = discount_factor * np.mean(payoffs)  # average discounted payoff
    std_error = discount_factor * np.std(payoffs) / np.sqrt(M)
    
    return option_price, std_error

for M in M_trials:
    optionPrices=[]
    stdErrors=[]
    price, error = monte_carlo_simulation(S0, K, T, r, sigma, M)
    optionPrices.append(price)
    stdErrors.append(error)
    option_prices.append(np.mean(optionPrices))
    std_errors.append(np.mean(stdErrors))

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

# Option Price Convergence
ax1.plot(M_trials, option_prices, marker='o')
ax1.set_xlabel('Number of Trials', fontsize=14)
ax1.set_ylabel('Option Price', fontsize=14)
ax1.set_title('Convergence of Option Price', fontsize=14)
ax1.tick_params(labelsize=12)
ax1.grid(True)

# Standard Error Convergence
ax2.plot(M_trials, std_errors, marker='o', color='red')
ax2.set_xlabel('Number of Trials', fontsize=14)
ax2.set_ylabel('Standard Error', fontsize=14)
ax2.set_yscale('log')
ax2.set_xscale('log')
ax2.set_title('Convergence of Standard Error', fontsize=14)
ax2.tick_params(labelsize=12)
ax2.grid(True)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig('Figures/4.1.1OptionPriceConvergence.png', dpi = 500)
plt.show()
print(option_prices)

#### Comparison with Assignment 1 Binomial Model
- Option Price Estimates: Assignment 1 provided a binomial tree model price of approximately 11.55 for a European Call option with specific parameters (1-year maturity, strike price of 99, risk-free rate of 6%, current stock price of 100, and volatility of 20%)​​. The Monte Carlo simulation generated option prices at different trial numbers as printed. When M = 50000, the option price calculated is about 11.54.
- Convergence Analysis: The binomial tree model showed convergence of option prices as the number of steps increased​. In Monte Carlo simulation, the standard error converges as the number of trials increases, showing increasing accuracy of the Monte Carlo estimate.
- Impact of Volatility: 

#### Varying Volality and Strike Price

In [ ]:
S0 = 100
T = 1.0 
r = 0.06
M = 100000    

strike_prices = [80, 90, 100, 110, 120]
volatilities = [0.1,0.2,0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

# Store the results
results = []

# Perform simulations for varying strike prices and volatilities
for K in strike_prices:
    for sigma in volatilities:
        price, error = monte_carlo_simulation(S0, K, T, r, sigma, M)
        results.append((K, sigma, price, error))

df = pd.DataFrame(results, columns=['Strike Price', 'Volatility', 'Option Price', 'Standard Error'])

pivot_price = df.pivot(index="Strike Price", columns="Volatility", values="Option Price")
pivot_error = df.pivot(index="Strike Price", columns="Volatility", values="Standard Error")

# Plotting heatmaps for Option Price and Standard Error
plt.figure(figsize=(13, 5))

plt.subplot(1, 2, 1)
sns.set(font_scale=1.1)
sns.heatmap(pivot_price, annot=True, fmt=".2f", linewidths=0.5)
plt.xlabel("Volatility", fontsize=16)
plt.ylabel("Strike Price", fontsize=16)
plt.title("Option Price (European Call)", fontsize=16)

plt.subplot(1, 2, 2)
sns.set(font_scale=1.1)
sns.heatmap(pivot_error, annot=True, fmt=".3f", linewidths=0.5)
plt.title("Standard Error", fontsize=16)
plt.xlabel("Volatility", fontsize=16)
plt.ylabel("Strike Price", fontsize=16)

plt.tight_layout()
plt.savefig('Figures/4.1.2OptionPriceHeatmap.png', dpi = 500)
plt.show()



### Q2: Heston stochastic volatility model

#### Implement the Heston stochastic volatility model using milstein method

In [ ]:
def milstein_scheme_for_heston(T, K, B, r, kappa, theta, xi, rho, delta_t, paths):
    n = int(T / delta_t)
    S = np.zeros((paths, n+1))
    v = np.zeros((paths, n+1))
    S[:, 0] = S0
    v[:, 0] = theta

    dw_v = np.random.normal(0, np.sqrt(delta_t), (paths, n))
    dw_s = rho * dw_v + np.sqrt(1 - rho**2) * np.random.normal(0, np.sqrt(delta_t), (paths, n))

    barrier_hit = np.zeros(paths, dtype=bool)  # Keep track of paths that hit the barrier

    for t in range(1, n+1):
        v[:, t] = v[:, t-1] + kappa * (theta - np.maximum(v[:, t-1], 0)) * delta_t + xi * np.sqrt(np.maximum(v[:, t-1], 0)) * dw_v[:, t-1]
        v[:, t] = np.maximum(v[:, t], 0)  # Take the positive part of the variance
        
        S[:, t] = S[:, t-1] * (1 + r * delta_t + np.sqrt(np.maximum(v[:, t-1], 0)) * dw_s[:, t-1]) + \
                  0.25 * xi ** 2 * (dw_s[:, t-1] ** 2 - delta_t)
                  
        # Apply the barrier
        barrier_hit = np.logical_or(barrier_hit, S[:, t] >= B)
    
    S[barrier_hit, :] = 0
    payoffs = np.maximum(S[:, -1] - K, 0)
    option_price = np.mean(payoffs) * np.exp(-r * T)
    
    return S, v, option_price, barrier_hit.sum()

In [ ]:
T = 1
K = 100
B = 120
r = 0.05
kappa = 2
theta = 0.04
xi = 0.1
rho = -0.7
delta_t = 1/252
S0 = 100

paths_range = np.arange(200, 10200,200)
option_prices = []
barrier_hits = []
standard_errors = []
for paths in paths_range:
    S_paths, v_paths, option_price, barrier_hit = milstein_scheme_for_heston(T, K, B, r, kappa, theta, xi, rho, delta_t, paths)
    option_prices.append(option_price)
    barrier_hits.append(barrier_hit.sum())
    payoffs = np.maximum(S_paths[:, -1] - K, 0)
    payoffs_std = np.std(payoffs, ddof=1) 
    standard_error = payoffs_std / np.sqrt(paths)
    standard_errors.append(standard_error)
    if paths == 10000:
        print(option_price, barrier_hit.sum(), standard_error)

fig, axs = plt.subplots(1, 3, figsize=(16, 5))

# Plot option prices
axs[0].plot(paths_range, option_prices, marker='o')
axs[0].set_title('Option Prices', fontsize=16)
axs[0].set_xlabel('Number of Paths', fontsize=16)
axs[0].set_ylabel('Option Price', fontsize=16)

# Plot barrier hits
axs[1].plot(paths_range, barrier_hits, marker='o', color='orange')
axs[1].set_title('Barrier Hits', fontsize=16)
axs[1].set_xlabel('Number of Paths', fontsize=16)
axs[1].set_ylabel('Count of Barrier Hits', fontsize=16)

# Plot standard errors
axs[2].plot(paths_range, standard_errors, marker='o', color='green')
axs[2].set_title('Standard Errors', fontsize=16)
axs[2].set_xlabel('Number of Paths', fontsize=16)
axs[2].set_ylabel('Standard Error', fontsize=16)

plt.tight_layout()
plt.tick_params(labelsize=14)
plt.savefig('Figures/4.2.1HestonOptionPriceConvergence.png', dpi = 500)


#### Varying parameters： the barrier B, correlation ρ and the vol-of-vol ξ parameters.

In [ ]:
T = 1 
K = 100
B = 120
B_range = np.arange(90, 500, 10)
r = 0.05 
kappa = 2
theta = 0.04
xi = 0.1 
xi_range = np.linspace(0.1, 0.99, 100)
rho = -0.7
rho_range = np.linspace(-0.9, 0.99, 100)
delta_t = 1/252
paths = 50000
S0 = 100 

plt.figure(figsize=(18, 6))

# Subplot 1: Varying B
plt.subplot(1, 3, 1)
option_prices_B = []
for b in B_range:
    _, _, option_price, _ = milstein_scheme_for_heston(T, K, b, r, kappa, theta, xi, rho, delta_t, paths)
    option_prices_B.append(option_price)
plt.plot(B_range, option_prices_B, marker='o')
plt.title('Option Price Sensitivity to Barrier Level B', fontsize=16)
plt.xlabel('Barrier Level B', fontsize=16)
plt.ylabel('Option Price', fontsize=16)

# Subplot 2: Varying xi
plt.subplot(1, 3, 2)
option_prices_xi = []
for x in xi_range:
    _, _, option_price, _ = milstein_scheme_for_heston(T, K, B, r, kappa, theta, x, rho, delta_t, paths)
    option_prices_xi.append(option_price)
plt.plot(xi_range, option_prices_xi, marker='o')
plt.title('Option Price Sensitivity to Vol of Vol ξ', fontsize=16)
plt.xlabel('Vol of Vol ξ', fontsize=16)
plt.ylabel('Option Price', fontsize=16)

# Subplot 3: Varying rho
plt.subplot(1, 3, 3)
option_prices_rho = []
for r in rho_range:
    _, _, option_price, _ = milstein_scheme_for_heston(T, K, B, r, kappa, theta, xi, r, delta_t, paths)
    option_prices_rho.append(option_price)
plt.plot(rho_range, option_prices_rho, marker='o')
plt.title('Option Price Sensitivity to Correlation ρ', fontsize=16)
plt.xlabel('Correlation ρ', fontsize=16)
plt.ylabel('Option Price', fontsize=16)

plt.tight_layout()
plt.show()
    

#### Option Price Sensitivity Analysis

In [ ]:
def milstein_scheme_for_heston(T, K, B, r, kappa, theta, xi, rho, delta_t, paths, S0, V0):
    n = int(T / delta_t)
    S = np.zeros((paths, n+1))
    v = np.zeros((paths, n+1))
    S[:, 0] = S0
    v[:, 0] = V0

    dw_v = np.random.normal(0, np.sqrt(delta_t), (paths, n))
    dw_s = rho * dw_v + np.sqrt(1 - rho**2) * np.random.normal(0, np.sqrt(delta_t), (paths, n))

    barrier_hit = np.zeros(paths, dtype=bool)

    for t in range(1, n+1):
        v[:, t] = np.maximum(v[:, t-1] + kappa * (theta - np.maximum(v[:, t-1], 0)) * delta_t + 
                             xi * np.sqrt(np.maximum(v[:, t-1], 0)) * dw_v[:, t-1], 0)
        S[:, t] = S[:, t-1] * (1 + r * delta_t + np.sqrt(np.maximum(v[:, t-1], 0)) * dw_s[:, t-1]) + \
                  0.25 * xi ** 2 * delta_t * (dw_s[:, t-1] ** 2 - 1)
        barrier_hit = np.logical_or(barrier_hit, S[:, t] >= B)
    
    S[barrier_hit, :] = 0
    payoffs = np.maximum(S[:, -1] - K, 0)
    option_price = np.mean(payoffs) * np.exp(-r * T)
    
    return S, v, option_price, barrier_hit
T = 1
K = 100
B = 120
r = 0.05
kappa = 2
theta = 0.04
xi = 0.1
rho = -0.7
delta_t = 1/252
paths = 10000
S0 = 100
V0 = theta  # Initial variance is set to long term variance theta
dS = 1  # Small change in asset price for Delta calculation
dv = 0.0004  # Small change in volatility for Vega and Vomma calculation
dT = 1/252  # Small change in time for Theta calculation

deltas = []
thetas = []
vegas = []
gammas = []
vommas = []

# Calculate the Greeks for each barrier level B
for B in B_range:
    # Calculate the original option price
    _, _, option_price_original, _ = milstein_scheme_for_heston(T, K, B, r, kappa, theta, xi, rho, delta_t, paths, S0, V0)
    
    # Delta
    _, _, option_price_bumped_S, _ = milstein_scheme_for_heston(T, K, B, r, kappa, theta, xi, rho, delta_t, paths, S0 + dS, V0)
    delta = (option_price_bumped_S - option_price_original) / dS
    deltas.append(delta)
    
    # Theta
    _, _, option_price_bumped_T, _ = milstein_scheme_for_heston(T - dT, K, B, r, kappa, theta, xi, rho, delta_t, paths, S0, V0)
    theta = (option_price_bumped_T - option_price_original) / dT
    thetas.append(theta)
    
    # Vega
    _, _, option_price_bumped_V, _ = milstein_scheme_for_heston(T, K, B, r, kappa, theta, xi, rho, delta_t, paths, S0, V0 + dv)
    vega = (option_price_bumped_V - option_price_original) / dv
    vegas.append(vega)
    
    # Gamma
    _, _, option_price_bumped_S2, _ = milstein_scheme_for_heston(T, K, B, r, kappa, theta, xi, rho, delta_t, paths, S0 + 2*dS, V0)
    gamma = (option_price_bumped_S2 - 2 * option_price_bumped_S + option_price_original) / dS**2
    gammas.append(gamma)
    
    # Vomma
    _, _, option_price_bumped_V2, _ = milstein_scheme_for_heston(T, K, B, r, kappa, theta, xi, rho, delta_t, paths, S0, V0 + 2*dv)
    # Calculate Vega for bumped volatility
    _, _, option_price_bumped_V, _ = milstein_scheme_for_heston(T, K, B, r, kappa, theta, xi, rho, delta_t, paths, S0, V0 + dv)
    # Calculate Vomma
    vomma = (option_price_bumped_V2 - 2 * option_price_bumped_V + option_price_original) / dv**2
    vommas.append(vomma)

# Create 1x5 subplot for each Greek
fig, axes = plt.subplots(1, 5, figsize=(25, 5))
# Plot Delta
axes[0].plot(B_range, deltas, marker='o')
axes[0].set_title('Delta Sensitivity to Barrier Level B', fontsize=16)
axes[0].set_xlabel('Barrier Level B', fontsize=14)
axes[0].set_ylabel('Delta', fontsize=14)
# Plot Theta
axes[1].plot(B_range, thetas, marker='o')
axes[1].set_title('Theta Sensitivity to Barrier Level B', fontsize=16)
axes[1].set_xlabel('Barrier Level B', fontsize=14)
axes[1].set_ylabel('Theta', fontsize=14)    
# Plot Vega
axes[2].plot(B_range, vegas, marker='o')
axes[2].set_title('Vega Sensitivity to Barrier Level B', fontsize=16)
axes[2].set_xlabel('Barrier Level B', fontsize=14)
axes[2].set_ylabel('Vega', fontsize=14)
# Plot Gamma
axes[3].plot(B_range, gammas, marker='o')
axes[3].set_title('Gamma Sensitivity to Barrier Level B', fontsize=16)
axes[3].set_xlabel('Barrier Level B', fontsize=14)
axes[3].set_ylabel('Gamma', fontsize=14)
# Plot Vomma
axes[4].plot(B_range, vommas, marker='o')
axes[4].set_title('Vomma Sensitivity to Barrier Level B', fontsize=16)
axes[4].set_xlabel('Barrier Level B', fontsize=14)
axes[4].set_ylabel('Vomma', fontsize=14)



## Part II Estimation of Sensitivities in MC

### Q1: Use bump-and-revalue method to estimate delta

## Part III Variance Reduction

### Q1: Compare the Asian call option result between the analytical expression and Monte-Carlo simulations

In [ ]:
S0 = 100 
K = 100   
T = 1      
r = 0.05     
sigma = 0.2   
N = 252        

def geometric_asian_call_price(S0, K, T, r, sigma, N):
    # Adjust volatility and interest rates based on the characteristics of Asian options
    sigma_hat = sigma * np.sqrt((2 * N + 1) / (6 * (N + 1)))
    r_hat = (r - 0.5 * sigma**2 + sigma_hat**2) / 2

    # Calculate d1 and d2
    d1 = (np.log(S0 / K) + (r_hat + 0.5 * sigma_hat**2) * T) / (sigma_hat * np.sqrt(T))
    d2 = d1 - sigma_hat * np.sqrt(T)

    # Calculate Asian call option prices based on the BS model
    geometric_asian_call_theoretical = S0 * np.exp((r_hat - r) * T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return geometric_asian_call_theoretical

geometric_asian_call_theoretical = geometric_asian_call_price(S0, K, T, r, sigma, N)
print(geometric_asian_call_theoretical)


In [ ]:
# Use Monte-Carlo simulation to compute Asian call option

S0 = 100       
K = 100        
T = 1          
r = 0.05       
sigma = 0.2    
N = 252       
n_simulations = 1000000  # Number of Monte Carlo simulations

# Simulate Brownian motion path
def simulate_gbm(S0, r, sigma, T, n_steps):
    dt = T/n_steps  
    dW = np.random.normal(scale=np.sqrt(dt), size=n_steps)
    log_S = np.log(S0) + np.cumsum((r - 0.5 * sigma**2) * dt + sigma * dW)
    S = np.exp(log_S)  
    return S

def asian_call_payoff(S_path, K, r, T):
    geometric_mean = np.exp(np.mean(np.log(S_path)))  
    payoff = np.maximum(geometric_mean - K, 0)        
    discounted_payoff = np.exp(-r * T) * payoff     
    return discounted_payoff



In [ ]:
np.random.seed()  
payoffs = np.zeros(n_simulations)

for i in range(n_simulations):
    S_path = simulate_gbm(S0, r, sigma, T, N)
    payoffs[i] = asian_call_payoff(S_path, K, r, T)

# Calculate the average payoff and hence the price of the option
Asian_call_Monte_Carlo = np.mean(payoffs)

print(f"Asian_call_Monte_Carlo = {Asian_call_Monte_Carlo}, Simulation time = {n_simulations}")
print(f"Asian_call_Analytical_result = {geometric_asian_call_theoretical}")
print(f"Relative error = {abs(Asian_call_Monte_Carlo - geometric_asian_call_theoretical)/geometric_asian_call_theoretical}")

### Q2 : Explain how the strategy of using this as a control variate works.

Control Variates Method

The **Control Variates Method** is a method used to reduce variance in Monte Carlo simulations. The core idea of this method is to use a control variable with a known expected value to reduce the variance of the simulation output. Here we use the value of an Asian option based on geometric averages as a control variable to reduce variance.

Specific Implementation Steps:

1. **Identify a Control Variate**: Select the geometric average Asian option as the control variate for the arithmetic average Asian option.

2. **Compute the Expected Value of the Control Variate**:  Calculate the expected value of the geometric average Asian option.

3. **Simulate Both Payoffs**: During the simulation, compute both the payoff of the option of interest (arithmetic average Asian option) and the payoff of the control variate (geometric average Asian option).

4. **Calculate the Covariance and Variance**: Estimate the covariance between the payoff and the control variate, and the variance of the control variate, from the simulated data.

5. **Adjust the Payoff**: Use the covariance and the known expected value of the control variate to adjust the payoff of the option of interest. The adjustment is typically made by constructing a linear combination of the payoff and the control variate. The coefficients of this linear combination are chosen to minimize the variance of the estimator.

6. **Estimate the Option Price**: Estimate the Option Price: Calculate the price of the option of interest using the adjusted payoffs.


### Q3：Apply the control variates technique

In [ ]:
geometric_asian_call_theoretical = geometric_asian_call_price(S0, K, T, r, sigma, N)

# Adding calculation of geometric mean
def asian_call_payoff_modified(S_path, K, r, T):
    arithmetic_mean = np.mean(S_path)
    geometric_mean = np.exp(np.mean(np.log(S_path)))
    payoff_arithmetic = np.maximum(arithmetic_mean - K, 0)
    payoff_geometric = np.maximum(geometric_mean - K, 0)
    discounted_payoff_arithmetic = np.exp(-r * T) * payoff_arithmetic
    discounted_payoff_geometric = np.exp(-r * T) * payoff_geometric
    return discounted_payoff_arithmetic, discounted_payoff_geometric

def control_variates_simulation(S0, K, T, r, sigma, N):
    payoffs_arithmetic = np.zeros(n_simulations)
    payoffs_geometric = np.zeros(n_simulations)

    # Simulation
    for i in range(n_simulations):
        S_path = simulate_gbm(S0, r, sigma, T, N)
        payoff_arithmetic, payoff_geometric = asian_call_payoff_modified(S_path, K, r, T)
        payoffs_arithmetic[i] = payoff_arithmetic
        payoffs_geometric[i] = payoff_geometric

    # Calculate the adjustment amount of the control variable
    cov_matrix = np.cov(payoffs_arithmetic, payoffs_geometric)
    cov_XY = cov_matrix[0, 1]
    var_Y = cov_matrix[1, 1]
    c = cov_XY / var_Y

    # Adjusting the simulated value of the arithmetic mean Asian call option using control variables
    adjusted_payoffs = payoffs_arithmetic - c * (payoffs_geometric - geometric_asian_call_theoretical)

    adjusted_price_estimate = np.mean(adjusted_payoffs)

    return np.mean(payoffs_arithmetic), adjusted_price_estimate
Basic_estimate, adjusted_estimate =  control_variates_simulation(S0, K, T, r, sigma, N)
print("Basic Arithmetic Mean Price Estimate:", Basic_estimate)
print("Adjusted Arithmetic Mean Price Estimate:", adjusted_estimate)

In [ ]:
# Run simulation 30 times
results_arithmetic = []
results_adjusted = []
simulation_time = 30
for _ in range(simulation_time):
    mean_arithmetic, adjusted_estimate = control_variates_simulation(S0, K, T, r, sigma, N)
    results_arithmetic.append(mean_arithmetic)
    results_adjusted.append(adjusted_estimate)

In [ ]:
# Box-plot
data = {'Basic Monte Carlo Method': results_arithmetic, 'Control Variates': results_adjusted}
data_long_format = pd.DataFrame(list(data.items()), columns=['Simulation method', 'Price Estimate']).explode('Price Estimate')
data_long_format['Price Estimate'] = data_long_format['Price Estimate'].astype(float)

plt.figure(figsize=(10, 6))
sns.boxplot(x='Simulation method', y='Price Estimate', data=data_long_format)
plt.title('Comparison of estimate result of two models', fontsize = 16)
plt.xlabel('Simulation method', fontsize = 15)
plt.ylabel('Price Estimate', fontsize = 15)
plt.grid(True)
plt.show()

print("Result under 30 simulation round")
print(f"For Basic Monte Carlo Method : mean = {np.mean(results_arithmetic)}, variance = {np.var(results_arithmetic)}")
print(f"For Control Variates Method : mean = {np.mean(results_adjusted)}, variance = {np.var(results_adjusted)}")